In [ ]:
# SENTIMENT ANALYSIS
# We will be cclassifying movie reviews (25,000) as either positive or negative
# each word is already encoded by an integer
# the num corresponds to its frquency > word 3 is the third most used

In [ ]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import numpy as np
import os

In [ ]:
VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data,test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
# review sample
train_data[0]
# the length of each review is oviously different

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

In [ ]:
# Due to different lengths we must make them the same length in order to fit them into our NN model
# our padding will be made of 0's
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [ ]:
# CREATING THE MODEL

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE,32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1,activation="sigmoid") # the sentiment is going to be [0,1]  ex. if <= 0.5 it will be a negative review
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# TRAINING

In [ ]:
# compiling the model
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])
# binary_crossentropy becuase we are trying to predict two things, either 0 or 1

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)
# 20% of the training data to evaluate the models we go through

Epoch 1/10
625/625 [==============================] - 38s 59ms/step - loss: 0.1215 - acc: 0.9610 - val_loss: 0.3437 - val_acc: 0.8626
Epoch 2/10
625/625 [==============================] - 48s 77ms/step - loss: 0.1015 - acc: 0.9663 - val_loss: 0.4257 - val_acc: 0.8636
Epoch 3/10
625/625 [==============================] - 38s 60ms/step - loss: 0.0860 - acc: 0.9731 - val_loss: 0.4581 - val_acc: 0.8704
Epoch 4/10
625/625 [==============================] - 37s 60ms/step - loss: 0.0694 - acc: 0.9793 - val_loss: 0.4657 - val_acc: 0.8730
Epoch 5/10
625/625 [==============================] - 38s 60ms/step - loss: 0.0605 - acc: 0.9818 - val_loss: 0.4182 - val_acc: 0.8784
Epoch 6/10
625/625 [==============================] - 36s 58ms/step - loss: 0.0514 - acc: 0.9845 - val_loss: 0.4164 - val_acc: 0.8722
Epoch 7/10
625/625 [==============================] - 37s 59ms/step - loss: 0.0431 - acc: 0.9883 - val_loss: 0.5064 - val_acc: 0.8746
Epoch 8/10
625/625 [==============================] - 37s 59ms

In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)
# accuracy of 86%

782/782 [==============================] - 11s 14ms/step - loss: 0.6666 - acc: 0.8557
[0.6665511131286621, 0.855679988861084]


In [ ]:
# MAKING PREDICTIONS

In [ ]:
# we need to encode any data that isn't already included in our dataset by using the same exact model
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens],MAXLEN)[0] # we want only the first entry

text = "That movie was amazing, just amazing"
encoded = encode_text(text)
print(encoded)

1641221/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [ ]:
# decode function

reverse_word_index = {value:key for (key,value) in word_index.items()}

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num] + " "

  return text[:-1]

print(decode_integers(encoded))

that movie was amazing just amazing


In [ ]:
# time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])


positive_review = "The movie was really good. The actors were great and the plot amazing. I definetly enjoyed it!"
negative_review = "It was awful"
meh_review = "The movie was kinda watchable. Good enough to spend some time."

predict(positive_review)
predict(negative_review)
predict(meh_review)
# we can see that the certain words greatly influence the results.
# Increase number of epochs?

1/1 [==============================] - 0s 21ms/step
[0.83985305]
1/1 [==============================] - 0s 20ms/step
[0.5188159]
1/1 [==============================] - 0s 21ms/step
[0.38517913]
